# Notebook to prepare CMIP6 data to obtain wind effect


CMIP6 data is provided by Dewi and downloaded from his work station: /nobackup/users/bars/CMIP6_regridded
For most models the data (uas and vas) was already available as a surface field but for some models 3D fields were downloaded whereof the surface was obtained: 'CESM2', 'CESM2-WACCM', 'CIESM', 'FGOALS-g3', 'NorESM2-LM', 'NorESM2-MM', 'TaiESM1'

In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import dask


dask.config.set(**{'array.slicing.split_large_chunks': False})

## Import and merge historical wind data

In [2]:
path = '/Volumes/Iris 300 GB/CMIP6'

### Import ua and va historical data

In [3]:
ua_his = xr.open_mfdataset(f'{path}/cmip6_ua_historical/*.nc').rename({'CorrectedReggrided_ua':'uas'})
va_his = xr.open_mfdataset(f'{path}/cmip6_va_historical/*.nc').rename({'CorrectedReggrided_va':'vas'})

### Import uas and vas historical data 

In [4]:
uas_his = xr.open_mfdataset(f'{path}/cmip6_uas_historical/*.nc').rename({'CorrectedReggrided_uas':'uas'})
vas_his = xr.open_mfdataset(f'{path}/cmip6_vas_historical/*.nc').rename({'CorrectedReggrided_vas':'vas'})

### Merge datasets

In [5]:
wind_his = xr.merge([ua_his, uas_his, va_his, vas_his])

## Import historical sea level data

In [6]:
zos_his = xr.open_mfdataset(f'{path}/cmip6_zos_historical/*.nc').rename({"CorrectedReggrided_zos":"zos"})

## Data preparations

### Select the same models for wind and sea level data

In [7]:
wind_his = wind_his.where(wind_his.model.isin(zos_his.model), drop = True)
zos_his = zos_his.where(zos_his.model.isin(wind_his.model), drop = True)

### Select data near tide gauge stations

In [8]:
stations = ['Vlissingen', 'Hoek v. Holland', 'Den Helder', 'Delfzijl', 'Harlingen', 'IJmuiden', 'Average']


def cmip6_np_coords(): 
    """
    Function to obtain a dataframe containing the coordinates of cmip6 nearest points to tide gauge models
    """
    lat_lst = [51.5, 52.5, 52.5, 53.5, 53.5, 52.5]
    lon_lst = [3.5, 4.5, 4.5, 6.5, 5.5, 4.5]
    df = pd.DataFrame({'station' : stations[:-1], 'lat' : lat_lst, 'lon' : lon_lst})
    df = df.set_index('station')
    
    return df

coord_df = cmip6_np_coords()


def obtain_data_near_tgstation(dataset):
    """
    Function to obtain a dataset containing timeseries near the Dutch tide gauges
        
    """
    
    
    lst = [] # List to save datasets
    
    # Loop over the tide gauge stations
    for index, row in coord_df.iterrows():
        
        lst.append(dataset.sel(lon = row.lon, lat = row.lat, method = 'nearest'))
        lst[-1] = lst[-1].drop(['lat', 'lon'])
        
    
    # Create a new dataset
    dataset_tg = xr.concat(lst, stations[:-1]).rename({'concat_dim':'station'})
    
    
    # Calculate average station
    average = dataset_tg.mean('station')
    average = average.assign_coords({'station':'Average'})
    
    
    # Concat to original dataarray
    dataset_tg = xr.concat([dataset_tg, average], dim='station')
    
    
    return dataset_tg
    

In [9]:
zos_his = obtain_data_near_tgstation(zos_his)
wind_his = obtain_data_near_tgstation(wind_his)

### Check if model variants match

In [10]:
for model in zos_his.model.data:
    print(zos_his.sel(model = model).zos.attrs['variant'], wind_his.sel(model = model).uas.attrs['variant'], wind_his.sel(model = model).vas.attrs['variant'])

r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r1i1p1f1 r1i1p1f1 r1i1p1f1
r

All variants are the same

### Obtain wind stress

In [11]:
wind_his = wind_his.assign(u2 = wind_his.uas*np.sqrt(wind_his.uas**2 + wind_his.vas**2))
wind_his = wind_his.assign(v2 = wind_his.vas*np.sqrt(wind_his.uas**2 + wind_his.vas**2))

wind_his = wind_his.drop(['uas', 'vas'])

## Check data

In [ ]:
for station in zos_his.station.data:
    print(station)
    zos_his.sel(station = station).zos.plot(col = 'model', col_wrap = 4)

In [ ]:
for station in wind_his.station.data:
    print(station)
    wind_his.sel(station = station).u2.plot(col = 'model', col_wrap = 4)

In [ ]:
for station in wind_his.station.data:
    print(station)
    wind_his.sel(station = station).v2.plot(col = 'model', col_wrap = 4)

## Remove FGOALS-g3 model as its wind data is weird

In [20]:
models = ['ACCESS-CM2', 'ACCESS-ESM1-5', 'BCC-CSM2-MR', 'BCC-ESM1', 'CAMS-CSM1-0',
       'CAS-ESM2-0', 'CESM2', 'CESM2-WACCM', 'CIESM', 'CMCC-CM2-SR5',
       'CMCC-ESM2', 'CNRM-CM6-1', 'CNRM-ESM2-1', 'CanESM5', 'CanESM5-CanOE',
       'EC-Earth3', 'EC-Earth3-AerChem', 'EC-Earth3-CC', 'EC-Earth3-Veg',
       'EC-Earth3-Veg-LR', 'FGOALS-f3-L', 'GFDL-CM4', 'GFDL-ESM4',
       'GISS-E2-1-G', 'GISS-E2-1-G-CC', 'GISS-E2-1-H', 'HadGEM3-GC31-LL',
       'HadGEM3-GC31-MM', 'INM-CM4-8', 'INM-CM5-0', 'IPSL-CM6A-LR',
       'MIROC-ES2L', 'MIROC6', 'MPI-ESM-1-2-HAM', 'MPI-ESM1-2-HR',
       'MPI-ESM1-2-LR', 'MRI-ESM2-0', 'NESM3', 'NorCPM1', 'NorESM2-LM',
       'NorESM2-MM', 'TaiESM1', 'UKESM1-0-LL']

In [21]:
zos_his = zos_his.where(zos_his.model.isin(models), drop = True)

In [23]:
wind_his = wind_his.where(wind_his.model.isin(models), drop = True)

## Save datasets

In [24]:
path = '/Users/iriskeizer/Documents/Zeespiegelscenarios/data'

In [25]:
zos_his.to_netcdf(f'{path}/sealevel_his.nc')

In [26]:
wind_his.to_netcdf(f'{path}/windstress_his.nc')

## Import and merge scenario wind data

In [57]:
path = '/Volumes/Iris 300 GB/CMIP6'

### Import ua and va scenario data

In [58]:
ua_ssp126 = xr.open_mfdataset(f'{path}/cmip6_ua_ssp126/*.nc').rename({'CorrectedReggrided_ua':'uas'})
va_ssp126 = xr.open_mfdataset(f'{path}/cmip6_va_ssp126/*.nc').rename({'CorrectedReggrided_va':'vas'})

ua_ssp245 = xr.open_mfdataset(f'{path}/cmip6_ua_ssp245/*.nc').rename({'CorrectedReggrided_ua':'uas'})
va_ssp245 = xr.open_mfdataset(f'{path}/cmip6_va_ssp245/*.nc').rename({'CorrectedReggrided_va':'vas'})

ua_ssp585 = xr.open_mfdataset(f'{path}/cmip6_ua_ssp585/*.nc').rename({'CorrectedReggrided_ua':'uas'})
va_ssp585 = xr.open_mfdataset(f'{path}/cmip6_va_ssp585/*.nc').rename({'CorrectedReggrided_va':'vas'})

### Import uas and vas scenario data 

In [59]:
uas_ssp126 = xr.open_mfdataset(f'{path}/cmip6_uas_ssp126/*.nc').rename({'CorrectedReggrided_uas':'uas'})
vas_ssp126 = xr.open_mfdataset(f'{path}/cmip6_vas_ssp126/*.nc').rename({'CorrectedReggrided_vas':'vas'})

uas_ssp245 = xr.open_mfdataset(f'{path}/cmip6_uas_ssp245/*.nc').rename({'CorrectedReggrided_uas':'uas'})
vas_ssp245 = xr.open_mfdataset(f'{path}/cmip6_vas_ssp245/*.nc').rename({'CorrectedReggrided_vas':'vas'})

uas_ssp585 = xr.open_mfdataset(f'{path}/cmip6_uas_ssp585/*.nc').rename({'CorrectedReggrided_uas':'uas'})
vas_ssp585 = xr.open_mfdataset(f'{path}/cmip6_vas_ssp585/*.nc').rename({'CorrectedReggrided_vas':'vas'})

### Merge datasets

In [60]:
wind_ssp126 = xr.merge([ua_ssp126, uas_ssp126, va_ssp126, vas_ssp126])
wind_ssp245 = xr.merge([ua_ssp245, uas_ssp245, va_ssp245, vas_ssp245])
wind_ssp585 = xr.merge([ua_ssp585, uas_ssp585, va_ssp585, vas_ssp585])

## Import scenario sea level data

In [61]:
zos_ssp126 = xr.open_mfdataset(f'{path}/cmip6_zos_ssp126/*.nc').rename({"CorrectedReggrided_zos":"zos"})
zos_ssp245 = xr.open_mfdataset(f'{path}/cmip6_zos_ssp245/*.nc').rename({"CorrectedReggrided_zos":"zos"})
zos_ssp585 = xr.open_mfdataset(f'{path}/cmip6_zos_ssp585/*.nc').rename({"CorrectedReggrided_zos":"zos"})

## Data preparations

### Only keep models that are in historical data

In [62]:
wind_ssp126 = wind_ssp126.where(wind_ssp126.model.isin(wind_his.model), drop = True)

wind_ssp245 = wind_ssp245.where(wind_ssp245.model.isin(wind_his.model), drop = True)

wind_ssp585 = wind_ssp585.where(wind_ssp585.model.isin(wind_his.model), drop = True)



### Select data near tide gauge stations

In [63]:
stations = ['Vlissingen', 'Hoek v. Holland', 'Den Helder', 'Delfzijl', 'Harlingen', 'IJmuiden', 'Average']


def cmip6_np_coords(): 
    """
    Function to obtain a dataframe containing the coordinates of cmip6 nearest points to tide gauge models
    """
    lat_lst = [51.5, 52.5, 52.5, 53.5, 53.5, 52.5]
    lon_lst = [3.5, 4.5, 4.5, 6.5, 5.5, 4.5]
    df = pd.DataFrame({'station' : stations[:-1], 'lat' : lat_lst, 'lon' : lon_lst})
    df = df.set_index('station')
    
    return df

coord_df = cmip6_np_coords()


def obtain_data_near_tgstation(dataset):
    """
    Function to obtain a dataset containing timeseries near the Dutch tide gauges
        
    """
    
    
    lst = [] # List to save datasets
    
    # Loop over the tide gauge stations
    for index, row in coord_df.iterrows():
        
        lst.append(dataset.sel(lon = row.lon, lat = row.lat, method = 'nearest'))
        lst[-1] = lst[-1].drop(['lat', 'lon'])
        
    
    # Create a new dataset
    dataset_tg = xr.concat(lst, stations[:-1]).rename({'concat_dim':'station'})
    
    
    # Calculate average station
    average = dataset_tg.mean('station')
    average = average.assign_coords({'station':'Average'})
    
    
    # Concat to original dataarray
    dataset_tg = xr.concat([dataset_tg, average], dim='station')
    
    
    return dataset_tg
    

In [64]:
zos_ssp126 = obtain_data_near_tgstation(zos_ssp126)
zos_ssp245 = obtain_data_near_tgstation(zos_ssp245)
zos_ssp585 = obtain_data_near_tgstation(zos_ssp585)


wind_ssp126 = obtain_data_near_tgstation(wind_ssp126)
wind_ssp245 = obtain_data_near_tgstation(wind_ssp245)
wind_ssp585 = obtain_data_near_tgstation(wind_ssp585)

### Obtain wind stress

In [65]:
wind_ssp126 = wind_ssp126.assign(u2 = wind_ssp126.uas*np.sqrt(wind_ssp126.uas**2 + wind_ssp126.vas**2))
wind_ssp126 = wind_ssp126.assign(v2 = wind_ssp126.vas*np.sqrt(wind_ssp126.uas**2 + wind_ssp126.vas**2))

wind_ssp245 = wind_ssp245.assign(u2 = wind_ssp245.uas*np.sqrt(wind_ssp245.uas**2 + wind_ssp245.vas**2))
wind_ssp245 = wind_ssp245.assign(v2 = wind_ssp245.vas*np.sqrt(wind_ssp245.uas**2 + wind_ssp245.vas**2))

wind_ssp585 = wind_ssp585.assign(u2 = wind_ssp585.uas*np.sqrt(wind_ssp585.uas**2 + wind_ssp585.vas**2))
wind_ssp585 = wind_ssp585.assign(v2 = wind_ssp585.vas*np.sqrt(wind_ssp585.uas**2 + wind_ssp585.vas**2))

wind_ssp126 = wind_ssp126.drop(['uas', 'vas'])
wind_ssp245 = wind_ssp245.drop(['uas', 'vas'])
wind_ssp585 = wind_ssp585.drop(['uas', 'vas'])

## Save datasets

In [68]:
path = '/Users/iriskeizer/Documents/Zeespiegelscenarios/data/cmip6/regression input'

In [69]:
zos_ssp126.to_netcdf(f'{path}/sealevel_ssp126.nc')
zos_ssp245.to_netcdf(f'{path}/sealevel_ssp245.nc')
zos_ssp585.to_netcdf(f'{path}/sealevel_ssp585.nc')

In [70]:
wind_ssp126.to_netcdf(f'{path}/windstress_ssp126.nc')
wind_ssp245.to_netcdf(f'{path}/windstress_ssp245.nc')
wind_ssp585.to_netcdf(f'{path}/windstress_ssp585.nc')